In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -U spacy
!pip install tensorflow

You should consider upgrading via the 'C:\Users\TANTAN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\TANTAN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf
pretrained_model = tf.keras.models.load_model('D:\Acer\hbi\model\machine_v2.keras')
# Show the model architecture
pretrained_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 128) │     96,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 128) │     92,416 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │    131,584 │ embedding[0][0],  │
│                     │ (None, 128),      │            │ not_equal[0][0]   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    131,584 │ embedding_1[0][0… │
│                     │ 128), (None,      │            │ lstm[0][1],       │
│                     │ 128), (None,      │            │ lstm[0][2]        │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 722) │     93,138 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,089,446 (4.16 MB)

 Trainable params: 544,722 (2.08 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 544,724 (2.08 MB)

In [4]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
import numpy as np
import spacy
from tensorflow.keras.preprocessing.sequence import pad_sequences
spacy_en = spacy.load('en_core_web_sm')
eos = '<eos>'
bos = '<bos>'

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [8]:
import pickle
with open('/content/drive/MyDrive/hbi/dictionary_en.pkl', 'rb') as f:
    en_vocabulary = pickle.load(f)
with open('/content/drive/MyDrive/hbi/dictionary_vi.pkl', 'rb') as f:
    vi_vocabulary = pickle.load(f)
with open('/content/drive/MyDrive/hbi/dictionary_vi_reverse.pkl', 'rb') as f:
    vi_vocabulary_reverse = pickle.load(f)
with open('/content/drive/MyDrive/hbi/col_no_translate') as f:
    no_translate_vocabulary = [line.replace('\n','') for line in f]

In [9]:
pretrained_emb_layer = pretrained_model.get_layer('embedding')
encoder_inputs = Input(shape=(None,))
encoder_emb = pretrained_emb_layer(encoder_inputs)  # Use the pre-trained layer
encoder_lstm = pretrained_model.get_layer('lstm')
_, state_h, state_c = encoder_lstm(encoder_emb)
encoder_states = [state_h, state_c]

In [10]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = pretrained_model.get_layer('embedding_1')
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm =pretrained_model.get_layer('lstm_1')
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = pretrained_model.get_layer('dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)
latent_dim=128
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [12]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vi_vocabulary[bos]

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''

    # greedy search
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # print(output_tokens)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # print(sampled_token_index)
        sampled_word =vi_vocabulary_reverse[sampled_token_index]
        decoded_sentence += ' '+ sampled_word

        # Exit condition: either hit max length or find stop character.
        if (sampled_word == eos or
           len(decoded_sentence) > 300):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        # Update states
        states_value = [h, c]
    return decoded_sentence


## PDF

In [ ]:
!pip install pymupdf
# !pip install googletrans==3.1.0a0
!pip install XlsxWriter

In [ ]:
import pymupdf
import numpy as np
import pandas as pd
# from googletrans import Translator, constants
from PIL import Image
import io
import re

In [ ]:
def fix_text(text):
  if isinstance(text, str):
    if text[:3]=='Col' or text[:3]=='Unn':
      text=""
    else:
      text=text.replace('\n', ' ')
  else:
    text=""
  return text

In [ ]:
def check_alpha(text):
   if text.isalpha():
     return True
   else:
     return False
def check_numeric(text):
    try:
        float(text)
        return True
    except ValueError:
        return False

def translate(seq_input):
  if type(seq_input)==str:
    seq_input = seq_input.lower()
  else:
    seq_input=str(seq_input)
  seq =[tok.text for tok in spacy_en.tokenizer(seq_input)]
  if len(seq)==1 and (not check_alpha(seq_input) or check_numeric(seq_input) or len(seq_input)==1):
    output = seq_input
  else:
    list_isalnum=[]
    input_arr = [1]
    input_arr_=[1]
    index_next=0
    enmaxlen=48 # max length of english sequence, get from source translate_en_vi.ipynb
    for text in seq:
      if text in en_vocabulary and not (check_numeric(text) or not check_alpha(text) or len(text)==1):
        input_arr.append(en_vocabulary[text])
        input_arr_.append(en_vocabulary[text])
      else:
        input_arr.append(en_vocabulary['<bos>'])
        list_isalnum.append(text)
        try:
          input_arr_.append(en_vocabulary[text])
        except:
          input_arr_.append(en_vocabulary['<bos>'])
    if 'i' in list_isalnum and 'd' in list_isalnum:
      list_isalnum.remove('i')
      list_isalnum.remove('d')
    # Translate
    if np.unique(input_arr).shape[0] == 1:
      output=seq_input
    else:
      input_arr_ = pad_sequences([input_arr_], maxlen = enmaxlen, padding = 'post')
      decoded_sentence = decode_sequence(input_arr_)
      out_str=decoded_sentence[:-5].replace('_', ' ').strip()
      out_list=out_str.split(' ')
      output=""
      arr=[]
      for index_item, item in enumerate(out_list):
        if check_alpha(item):
          output+=item+" "
        else:
          arr.append(index_item)
      for i in list_isalnum:
        spaces_indices = [match.start() for match in re.finditer(' ', output)]
        if spaces_indices:
          try:
            index=arr.pop(0)-1
          except:
            index=0
          output=output[:spaces_indices[index]]+" "+i+output[spaces_indices[index]:]
        else:
          output += " " + i
  return output

In [ ]:
# translator = Translator()
col_name_tranl=["Product Type","Piece name"]
def trans_df(df):
  name_col=[]
  list_col=[col for col in df]
  if list_col[0] in col_name_tranl:
    name_col=[translate(fix_text(col)) for col in list_col]
  else:
    name_col=[fix_text(col) for col in list_col]
  for col in list_col:
    for row in range(len(df[col])):
      df.loc[row,col]=fix_text(df.loc[row,col])
      if df.loc[row,col] in no_translate_vocabulary :
        if df.loc[row,col][0]!="(":
          df.loc[row,col]=translate(df.loc[row,col])
        else:
          df.loc[row,col]=df.loc[row,col]
        break
      else:
        df.loc[row,col]=translate(df.loc[row,col])
  df2=df.set_axis(name_col, axis=1)
  return df2

In [ ]:
doc = pymupdf.open("/content/drive/MyDrive/hbi/test.pdf")
with pd.ExcelWriter('pdf_output.xlsx') as writer:
    for index_page in range(len(doc)):
        page = doc[index_page]
        tabs = page.find_tables(strategy='lines_strict')
        for table_index, table in enumerate(tabs):
            # Convert pymupdf Table to Pandas DataFrame
            df = table.to_pandas()
            # df.to_excel(writer, sheet_name=f'Sheet{index_page}_{table_index+1}')
            result = trans_df(df)
            result.to_excel(writer, sheet_name=f'Sheet{index_page}_{table_index+1}',index=False)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━

.

In [ ]:
data_excel=pd.read_excel('/content/drive/MyDrive/hbi/Chuyển từ PDF_lines-strict.xlsx',sheet_name=None)

In [ ]:
for i in data_excel:
  print(i)

Sheet0_1
Sheet0_2
Sheet1_1
Sheet2_1
Sheet3_1
Sheet4_1
Sheet5_1
Sheet6_1
Sheet13_1
Sheet14_1
Sheet15_1
Sheet16_1
Sheet17_1
Sheet18_1
Sheet19_1
Sheet20_1
Sheet21_1
Sheet22_1
Sheet23_1
Sheet24_1
Sheet25_1
Sheet26_1
Sheet27_1
Sheet28_1
Sheet29_1
Sheet30_1
Sheet31_1
Sheet32_1
Sheet33_1
Sheet34_1
Sheet35_1
Sheet36_1
Sheet37_1


In [ ]:
with pd.ExcelWriter('pdf_output.xlsx') as writer:
    for index_sheet in data_excel:
        df_sheet = data_excel[index_sheet].drop(("Unnamed: 0"), axis=1)
        result=trans_df(df_sheet)
        result.to_excel(writer, sheet_name=index_sheet) # Use index to differentiate sheets from different pages

### Spire

In [ ]:
!pip install spire.pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 18.2 MB/s eta 0:00:00


In [ ]:
from spire.pdf import *
from spire.pdf.common import *
pdf = PdfDocument()
pdf.LoadFromFile("/content/drive/MyDrive/hbi/test.pdf")
data_excel=pd.read_excel('/content/drive/MyDrive/hbi/Chuyển từ PDF_lines-strict.xlsx',sheet_name=None)
for index_sheet in data_excel:
  df_sheet = data_excel[index_sheet].drop(("Unnamed: 0"), axis=1)
  index_page=int(index_sheet[5:-2])
  page_pdf = pdf.Pages.get_Item(index_page)
  # Create an object of PdfTextReplacer class
  replacer = PdfTextReplacer(page_pdf)
  for col in df_sheet:
    for row in range(len(df_sheet[col])):
      text=df_sheet.loc[row,col]
      translated_text = translate(check_text(df_sheet.loc[row,col]))
      if text and translated_text:
        replacer.ReplaceText(str(text), str(translated_text))
pdf.SaveToFile("ReplaceFirstMatch.pdf")
pdf.Close()

1/1 [==============================] - 0s 52ms/step


SpireException: Arg_NullReferenceException:   at sprdv8..ctor(Byte[], String) + 0x1c8
   at sprdv2.spra3u() + 0x117
   at sprdvp.spra3h() + 0x1b
   at sprdn4.spra(sprdvl) + 0x61
   at Spire.Pdf.Texts.PdfTextState.spra(PdfTextState, sprdod) + 0x73
   at Spire.Pdf.Texts.PdfTextState..ctor(sprdn6) + 0x64
   at sprdn6.spra(sprdob, Int32, Int32, PdfTextReplaceOptions) + 0xf9
   at sprdn5.sprb(String) + 0x166
   at Spire.Pdf.Texts.PdfTextReplacer.ReplaceText(String, String) + 0xb4
   at Spire.Pdf.AOT.NLPdfTextReplacer.PdfTextReplacer_ReplaceText(IntPtr, IntPtr, IntPtr, IntPtr) + 0x83

In [ ]:
from spire.pdf.common import *
from spire.pdf import *

# Create an object of the PdfDocument class
doc = PdfDocument()
# Load a PDF file
doc.LoadFromFile("Adobe Acrobat.pdf")

# Get the first page
page = doc.Pages[0]

# Iterate through the pages in the document
for i in range(doc.Pages.Count):
    # Get the current page
    page = doc.Pages[i]
    # Create an object of the PdfTextReplace class and pass the page to the constructor of the class as a parameter
    replacer = PdfTextReplacer(page)

    # Replace the first instance of a specific text with new text
    replacer.ReplaceText("Adobe Acrobat", "PDF Editor")

# Save the resulting file
doc.SaveToFile("ReplaceFirstFoundText.pdf")
doc.Close()

## Get image from PDF

In [ ]:
for index_page in range(len(doc)):
      imgs = page.get_images(full=True) #  If you want to get the raw image data, use full=True
      for image_index, img in enumerate(imgs):
          # get the XREF of the image
          xref = img[0]
          # extract the image bytes
          base_image = doc.extract_image(xref) # Use extract_image() from PyMuPDF
          image_bytes = base_image["image"]
          # get the image extension
          image_ext = base_image["ext"]
          # load it to PIL
          image = Image.open(io.BytesIO(image_bytes))
          # save it to local disk
          image.save(open(f"image{page.number+1}_{image_index}.{image_ext}", "wb")) # Use page.number to get the page number

## Test

In [ ]:
for index_page in range(2):
    imgs = page.get_images(full=True) #  If you want to get the raw image data, use full=True
    for image_index, img in enumerate(imgs):
        # get the XREF of the image
        xref = img[0]
        # extract the image bytes
        base_image = doc.extract_image(xref) # Use extract_image() from PyMuPDF
        image_bytes = base_image["image"]
        # get the image extension
        image_ext = base_image["ext"]
        # load it to PIL
        image = Image.open(io.BytesIO(image_bytes))
        # save it to local disk
        image.save(open(f"image{page.number+1}_{image_index}.{image_ext}", "wb")) # Use page.number to get the page number

NameError: name 'page' is not defined

In [ ]:
from openpyxl import Workbook
from openpyxl import load_workbook
wb= load_workbook('/content/output.xlsx')
ws=wb.active
from openpyxl.drawing.image import Image
img = Image('/content/image2_0.jpeg')
ws.add_image(img,'A1')
wb.save('/content/output.xlsx')

# Define a folder to save images (optional)
# output_folder = "extracted_images"

# # Create the folder if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# # Save the image
# image_path = f"{output_folder}/page_{page_index+1}_image_{image_index}.{image_ext}"
# with open(image_path, "wb") as f:
#     f.write(image_bytes)
# print(f"Image saved to: {image_path}")



In [ ]:
pdf_file = pymupdf.open("/content/drive/MyDrive/hbi/test.pdf") # Open the PDF using fitz to extract images
page = pdf_file[2]
# imgs_list = page.get_images() # Use get_images() from PyMuPDF
imgs_list = page.get_images(full=True) #  If you want to get the raw image data, use full=True
for image_index, img in enumerate(imgs_list):
  # get the XREF of the image
  xref = img[0]
  # extract the image bytes
  base_image = pdf_file.extract_image(xref) # Use extract_image() from PyMuPDF
  image_bytes = base_image["image"]
  # get the image extension
  image_ext = base_image["ext"]
  # load it to PIL
  image = Image.open(io.BytesIO(image_bytes))
  # save it to local disk
  image.save(open(f"image{page.number+1}_{image_index}.{image_ext}", "wb")) # Use page.number to get the page number

In [ ]:
doc = pymupdf.open("/content/drive/MyDrive/hbi/test.pdf")
page = doc[1]
tabs = page.find_tables(strategy='lines_strict')
print(f"{len(tabs.tables)} table(s) on {page}")
tab = tabs[0]
df = tab.to_pandas()
df
col_name_tranl=["Product Type","Piece name"]
name_col=[]
list_name=[col for col in df]
print(list_name)
if list_name[0] in col_name_tranl:
  name_col=[translate(fix_text(col)) for col in list_name]
else:
  name_col=[fix_text(col) for col in list_name]

print(name_col)
# for col in list_name[1:]:
#     if col in col_name_tranl:
#       # name_col.append(translator.translate(check_text(col), dest='vi').text)
#       name_col.append(translate(fix_text(col)))
#     else:


1 table(s) on page 1 of /content/drive/MyDrive/hbi/test.pdf
['Product Type', 'Spec', 'Change 1', 'Change 2', 'Change 3', 'Comments', 'Date Modified', 'Last Edited By']
1/1 [==============================] - 0s 28ms/step
['loại sản phẩm ', 'thông số kỹ thuật ', 'thay 1 đổi ', 'thay đổi 2 ', 'thay 3 đổi ', 'bình luận ', 'ngày sửa đổi ', 'chỉnh sửa lần cuối bởi ']


In [ ]:
translator = Translator()
name_col=[]
for i in df:
  name_col.append(translator.translate(i, dest='vi').text)
  for j in range(len(df[i])):
    df[i][j]=translator.translate(df[i][j], dest='vi').text
print(name_col)
df2=df.set_axis(name_col, axis=1)
df2

['Loại sản phẩm', 'thông số kỹ thuật', 'Thay đổi 1', 'Thay đổi 2', 'Thay đổi 3', 'Bình luận', 'Ngày sửa đổi', 'Chỉnh sửa lần cuối bởi']


,Loại sản phẩm,thông số kỹ thuật,Thay đổi 1,Thay đổi 2,Thay đổi 3,Bình luận,Ngày sửa đổi,Chỉnh sửa lần cuối bởi
0,MẪU,005,Trang hình ảnh đánh dấu được cập nhật,,,Đã thêm ID điểm đánh dấu mới T148H4 & T148H5 t...,12/06/2023,Huong.An
1,MẪU,005,Đã hoàn thành biện pháp. Thay đổi,,,thông số kỹ thuật được cập nhật lên phiên bản ...,12/05/2023,kill.kasuriachillage
2,QUẦN ÁO,004,Cắt thay đổi BOM,,,Cập nhật chiều rộng cắt,05/24/2023,Giang.Doan
3,MẪU,005,Trang hình ảnh đánh dấu được cập nhật,,,Cập nhật cách sử dụng T148H3L; T148H3M; Đế dự ...,05/22/2023,Huong.An
4,MẪU,005,Đã hoàn thành biện pháp. Thay đổi,,,Đã cập nhật chiều dài mở cánh bên trong để tín...,05/10/2023,Kimberly.Yontz
5,QUẦN ÁO,003,Bổ sung thực vật,,,Thêm nhà máy may 95 cho tất cả các mfgs trong ...,04/19/2023,HuongThi.Phung1
6,QUẦN ÁO,003,Thay đổi mẫu,,,Tiết kiệm chi phí 2022 - Mẫu cập nhật lên T148...,11/10/2022,Nam.Le1
7,MẪU,003,Đã hoàn thành biện pháp. Thay đổi,,,Độ mở cửa bay giảm đi ¼” và được đổi tên thành...,10/19/2022,Giang.Doan
8,MẪU,005,Thay đổi mẫu,,,"Giảm độ mở của đường viền đi 1/4"" + giảm chiều...",10/19/2022,Giang.Doan
9,QUẦN ÁO,002,,Bổ sung đường màu,,Đã thêm PGU & IMG màu vào S22,04/29/2022,Nam.Le1


In [ ]:
df2=df.to_numpy()
df2
# result = trans_text(df2,'vi')
# print("Kết quả")
# pd.DataFrame(result)

## Excel

In [ ]:
dataframe=pd.read_excel('/content/drive/MyDrive/hbi/7349K0 7349B0 MRL Ringspun Wicking Fashion Boxer Brief Pkg Spec (1).xlsx',sheet_name=None)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [ ]:
for i in dataframe:
  print(i)

Style Summary
Routing
7349K0
7349B0
Pack Order Link
PKG BOMs (JB&BAG) pack1
How to FOLD pack2
UWMCH1E Horizontal 3x1.5 EPC
 CASE Page S-XL
 CASE Page 2XL-3XL
PUT UP CODE 
PACKAGING HISTORY-pack5


In [ ]:
dataframe['Style Summary'].head(2)

,HANESBRANDS INC.PRODUCT SPECIFICATION - Packaging,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Selling style #:,"7349K0, 7349B0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
with pd.ExcelWriter('excel_output.xlsx') as writer:
    for index_sheet in dataframe:
        df_sheet = dataframe[index_sheet]
        result=trans_df(df_sheet)
        result.to_excel(writer, sheet_name=index_sheet) # Use index to differentiate sheets from different pages

1/1 [==============================] - 0s 25ms/step


<ipython-input-69-fb6c916c9af1>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row,col]=fix_text(df.loc[row,col])


1/1 [==============================] - 0s 26ms/step


Get image from excel

In [ ]:
!pip install Spire.XLS

In [ ]:
from spire.xls import *
from spire.xls.common import *
workbook= Workbook()
workbook.LoadFromFile('/content/drive/MyDrive/hbi/7349K0 7349B0 MRL Ringspun Wicking Fashion Boxer Brief Pkg Spec (1).xlsx')
workbook

In [ ]:
for index_sheet in range(workbook.Worksheets.Count):
  sheet=workbook.Worksheets[index_sheet]
  for index_picture in range(sheet.Pictures.Count):
    excelPicture=sheet.Pictures[index_picture]
    excelPicture.Picture.Save(f"image{index_sheet}_{index_picture}.png")